In [10]:
import sagemaker
import boto3
import pandas as pd
import subprocess 

# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token, region_name = "eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

credentials =  {
    "boto_session": session,
    "sagemaker_session": sagemaker_session,
    "role": role
}

In [12]:
# FAISS vector store
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vs = FAISS.load_local("faiss", hf_embedding,allow_dangerous_deserialization=True)

In [13]:
bucket = sagemaker_session.default_bucket()
prefix = 'faiss-deploy'
s3_uri = f's3://{bucket}/{prefix}'


In [15]:
!tar -czvf model.tar.gz faiss
!tar -tvf model.tar.gz
model_uri = sagemaker_session.upload_data('model.tar.gz', bucket = bucket, key_prefix=f"{prefix}/model")

faiss/
faiss/index.pkl
faiss/index.faiss
drwxrwxr-x willem/willem     0 2024-08-07 20:31 faiss/
-rw-rw-r-- willem/willem 214896323 2024-08-07 20:58 faiss/index.pkl
-rw-rw-r-- willem/willem  70800429 2024-08-07 20:58 faiss/index.faiss


In [11]:
# Create sagemaker model
import time
image = sagemaker.image_uris.retrieve(
    framework='pytorch',
    region='eu-west-1',
    image_scope='inference',
    version='1.12',
    instance_type='ml.m5.2xlarge',
)

In [12]:
model_name = f'faiss-vs-{int(time.time())}'
faiss_model_sm = sagemaker.model.Model(
    model_data=model_uri,
    image_uri=image,
    role=role,
    entry_point="inference.py",
    source_dir='src',
    name=model_name,
)

In [13]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name = f'faiss-endpoint-{int(time.time())}'
faiss_model_sm.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.2xlarge",
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    wait=True,
)

ClientError: An error occurred (AccessDenied) when calling the CreateBucket operation: Access Denied